In [3]:
# Core 
import datetime
import os
import glob
from IPython.display import HTML, Image, display
import tempfile
import shutil

# Analysis 
import xarray as xr
import numpy as np
import pyproj as pp
import scipy as sp
import metpy as mp

# Plotting 
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.ticker as mticker
from matplotlib.animation import FuncAnimation
import matplotlib.colors as colors

# Debugging 
import pdb
%load_ext line_profiler

# Radar Tools
import pyart
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate
from tint.visualization import embed_mp4_as_gif

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [41]:
# Store filenames for an example day of data
filenames = sorted(glob.glob('/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161220/*.nc'))
grids_low = (pyart.io.read_grid(fn, include_fields = 'reflectivity') for fn in filenames)
grids_mid = (pyart.io.read_grid(fn, include_fields = 'reflectivity') for fn in filenames)

# Specify bounds of CPOL 1000 m gridded dataset
lon_min = 130.40
lon_max = 131.69

lat_max = -11.61
lat_min = -12.87

# Define settings for low and mid level tracking
settings_low = {
    'MIN_SIZE' : 100, 
    'FIELD_THRESH' : 40,
    'GS_ALT' : 1500,
}

settings_mid = {
    'MIN_SIZE' : 100, 
    'FIELD_THRESH' : 40,
    'GS_ALT' : 5000,
}

# Calculate high and low level tracks
tracks_obj_low  = Cell_tracks()
tracks_obj_mid  = Cell_tracks()

for parameter in ['MIN_SIZE', 'FIELD_THRESH', 'GS_ALT']:
    tracks_obj_low.params[parameter] = settings_low[parameter]
    tracks_obj_mid.params[parameter] = settings_mid[parameter]

# Calculate tracks
tracks_obj_low.get_tracks(grids_low)
tracks_obj_mid.get_tracks(grids_mid)

Writing tracks for scan 0
No cells found in scan 1
No cells found in scan 2
No cells found in scan 3
Writing tracks for scan 4
Writing tracks for scan 5
Writing tracks for scan 6
No cells found in scan 7
No cells found in scan 8
Writing tracks for scan 9
No cells found in scan 10
No cells found in scan 11
Writing tracks for scan 12
Writing tracks for scan 13
Writing tracks for scan 14
Writing tracks for scan 15
Writing tracks for scan 16
Writing tracks for scan 17
Writing tracks for scan 18
Writing tracks for scan 19
Writing tracks for scan 20
No cells found in scan 21
Writing tracks for scan 22
Writing tracks for scan 23
Writing tracks for scan 24
Writing tracks for scan 25
Writing tracks for scan 26
Writing tracks for scan 27
Writing tracks for scan 28
Writing tracks for scan 29
No cells found in scan 30
No cells found in scan 31
No cells found in scan 32
No cells found in scan 33
No cells found in scan 34
No cells found in scan 35
No cells found in scan 36
No cells found in scan 37


In [47]:
tracks_obj_mid.params

{'FIELD_THRESH': 40,
 'MIN_SIZE': 100,
 'SEARCH_MARGIN': 4000,
 'FLOW_MARGIN': 10000,
 'MAX_FLOW_MAG': 50,
 'MAX_DISPARITY': 999,
 'MAX_SHIFT_DISP': 15,
 'ISO_THRESH': 8,
 'ISO_SMOOTH': 3,
 'GS_ALT': 5000}

In [42]:
tracks_obj_low.tracks

,,time,grid_x,grid_y,lon,lat,area,vol,max,max_alt,isolated
scan,uid,,,,,,,,,,
0,0,2016-12-20 02:40:05,28.694,116.972,130.6677,-11.8260,108.0,262.5,49.198586,5.5,False
4,1,2016-12-20 03:20:02,93.614,64.228,131.2654,-12.3028,127.0,345.0,46.534042,6.0,False
5,1,2016-12-20 03:30:02,93.932,56.012,131.2654,-12.3747,162.0,432.0,47.399334,6.5,False
6,1,2016-12-20 03:40:06,94.486,50.400,131.2655,-12.4287,105.0,242.5,46.362911,5.5,False
9,2,2016-12-20 04:10:02,103.375,15.442,131.3487,-12.7433,104.0,198.5,47.872238,5.0,False
12,3,2016-12-20 04:50:02,19.723,130.819,130.5852,-11.6999,155.0,380.5,52.621647,6.5,False
13,3,2016-12-20 05:00:02,27.120,124.394,130.6494,-11.7630,142.0,383.0,52.597782,5.5,False
14,3,2016-12-20 05:10:02,34.892,119.269,130.7229,-11.8080,130.0,359.0,51.285446,5.0,False
15,4,2016-12-20 05:20:02,54.706,14.154,130.9061,-12.7525,136.0,432.5,50.675404,6.5,False


In [44]:
tracks_obj_mid.tracks

,,time,grid_x,grid_y,lon,lat,area,vol,max,max_alt,isolated
scan,uid,,,,,,,,,,
0,0,2016-12-20 02:40:05,28.694,116.972,130.6677,-11.8260,108.0,262.5,49.198586,5.5,False
4,1,2016-12-20 03:20:02,93.614,64.228,131.2654,-12.3028,127.0,345.0,46.534042,6.0,False
5,1,2016-12-20 03:30:02,93.932,56.012,131.2654,-12.3747,162.0,432.0,47.399334,6.5,False
6,1,2016-12-20 03:40:06,94.486,50.400,131.2655,-12.4287,105.0,242.5,46.362911,5.5,False
9,2,2016-12-20 04:10:02,103.375,15.442,131.3487,-12.7433,104.0,198.5,47.872238,5.0,False
12,3,2016-12-20 04:50:02,19.723,130.819,130.5852,-11.6999,155.0,380.5,52.621647,6.5,False
13,3,2016-12-20 05:00:02,27.120,124.394,130.6494,-11.7630,142.0,383.0,52.597782,5.5,False
14,3,2016-12-20 05:10:02,34.892,119.269,130.7229,-11.8080,130.0,359.0,51.285446,5.0,False
15,4,2016-12-20 05:20:02,54.706,14.154,130.9061,-12.7525,136.0,432.5,50.675404,6.5,False


In [48]:
tracks_obj_mid.tracks.xs('1', level='uid')

,time,grid_x,grid_y,lon,lat,area,vol,max,max_alt,isolated
scan,,,,,,,,,,
4,2016-12-20 03:20:02,93.614,64.228,131.2654,-12.3028,127.0,345.0,46.534042,6.0,False
5,2016-12-20 03:30:02,93.932,56.012,131.2654,-12.3747,162.0,432.0,47.399334,6.5,False
6,2016-12-20 03:40:06,94.486,50.400,131.2655,-12.4287,105.0,242.5,46.362911,5.5,False


In [58]:
tracks_obj_low.tracks

,,time,grid_x,grid_y,lon,lat,area,vol,max,max_alt,isolated
scan,uid,,,,,,,,,,
0,0,2016-12-20 02:40:05,28.694,116.972,130.6677,-11.8260,108.0,262.5,49.198586,5.5,False
4,1,2016-12-20 03:20:02,93.614,64.228,131.2654,-12.3028,127.0,345.0,46.534042,6.0,False
5,1,2016-12-20 03:30:02,93.932,56.012,131.2654,-12.3747,162.0,432.0,47.399334,6.5,False
6,1,2016-12-20 03:40:06,94.486,50.400,131.2655,-12.4287,105.0,242.5,46.362911,5.5,False
9,2,2016-12-20 04:10:02,103.375,15.442,131.3487,-12.7433,104.0,198.5,47.872238,5.0,False
12,3,2016-12-20 04:50:02,19.723,130.819,130.5852,-11.6999,155.0,380.5,52.621647,6.5,False
13,3,2016-12-20 05:00:02,27.120,124.394,130.6494,-11.7630,142.0,383.0,52.597782,5.5,False
14,3,2016-12-20 05:10:02,34.892,119.269,130.7229,-11.8080,130.0,359.0,51.285446,5.0,False
15,4,2016-12-20 05:20:02,54.706,14.154,130.9061,-12.7525,136.0,432.5,50.675404,6.5,False
